# 1. set up

## 1.1. libraries

In [ ]:
import sys
print("print version")
print(sys.version)

import os
import time

import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import gc # for collecting garbage
import math # for plotting network

import matplotlib.gridspec as gridspec # for plotting figure
import matplotlib.patches as mpatches # for plotting network
from matplotlib.lines import Line2D # for plotting network
from matplotlib.patches import Circle # for plotting network
from matplotlib.patches import Patch # for plotting network
from matplotlib.patches import Rectangle # for plotting figure
import matplotlib.patches as patches # for plotting figure

import pickle

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from helper_plot import *

## 1.2. load up variables

In [ ]:
path_project = '../'
path_plot = path_project + 'figure/'
path_meta = path_project + 'data/population/'

df_meta = pd.read_csv(path_meta + 'meta_merged.csv')
df_desc = pd.read_csv(path_meta + 'desc_subpopulation.tsv', delimiter = '\t')

df_desc = df_desc[['Population code', 'Population description']]
df_desc.set_index('Population code', inplace = True)
dict_desc = df_desc['Population description'].to_dict()
n_PC = 600

df_meta_sorted = df_meta.sort_values(['POP', 'SUP']).reset_index()

df_meta_unique = df_meta[['SUP', 'POP']].drop_duplicates()
vec_label_sorted = df_meta_sorted.loc[0:3200,:][['SUP','POP']].drop_duplicates()['SUP'].values
vec_label_super_sorted = df_meta_sorted.loc[0:3200,:][['POP']].drop_duplicates()['POP'].values


# setting for plotting
para_scatter_fig2 = {
    'marker': 'o',
    'alpha': 0.25,
    'lw': 0
}

vec_name_data = ['train_test', 'full']

# read eigenvalues and vectors
path_eigen = {}
path_eigen['train_test'] = path_project + 'data/eigen/train/'
path_eigen['full'] = path_project + 'data/eigen/center_full_4.5M/'

e_val = {}
e_vec = {}

# PC for traning data
mat_PC = {}

for name in vec_name_data:
    e_val[name] = pd.read_csv(path_eigen[name] + 'eigenvalues_4.5M.tsv', sep = '\t', header = None).to_numpy().flatten()
    e_vec[name] = pd.read_csv(path_eigen[name] + 'eigenvectors_4.5M.tsv', sep = '\t', header = None).to_numpy()

    mat_PC[name] = e_vec[name] * (np.sqrt(e_val[name]).reshape(1, -1))
    mat_PC[name] = mat_PC[name][:, 0:2560]

# read labels
path_index = {}
path_index['train_test'] = path_project + 'data/label/'

col_index_train = {}
col_index_train['train_test'] = pd.read_csv(path_index['train_test'] + 'index_train.csv', 
                                      sep = '\t', header = None).to_numpy().flatten() - 3
col_index_test = {}
col_index_test['train_test'] = pd.read_csv(path_index['train_test'] + 'index_test.csv', 
                                     sep = '\t', header = None).to_numpy().flatten() - 3

label = list(df_meta[df_meta['SUP'] != 'IBS,MSL']['SUP'])

# get y-label
path_label = path_project + 'data/population/'
df_label = pd.read_csv(path_label + 'meta_3202.csv', sep = ',')
label_train = {}
label_test = {}
label_train_super = {}
label_train_super['full'] = np.array(df_label['POP'][list(range(0, 788)) + list(range(789, 3202))])
label_train_super['train_test'] = np.array(df_label['POP'][col_index_train['train_test'] - 1])

label_train['full'] = np.array(df_label['SUP'][list(range(0, 788)) + list(range(789, 3202))])
label_train['train_test'] = np.array(df_label['SUP'][col_index_train['train_test'] - 1])
label_test['train_test'] = np.array(df_label['SUP'][col_index_test['train_test'] - 1])

# read test data
## takes about 1 minute
path_test = {}
path_test['train_test'] = path_project + 'data/mat_PC/test/'
now = time.time()
vec_name = ['full', '500k', '50k', '5k']
vec_name_45M = [i + '_4.5M' for i in vec_name]
vec_scale_PC = [1, n_mut / 5e5, n_mut / 5e4, n_mut / 5e3]
mat_PC_test = {}

# 240310 WORKING ON THE PART BELOW
# new versions at /sahandlab/shong/project_AISNP/data/mat_PC/test_missing/90.0/ # DELETE
vec_name_wo_45M = ['full', '500k', '50k', '5k']

for i, name in enumerate(vec_name_45M):
    mat_temp = pd.read_csv(f'{path_test["train_test"]}mat_PC_test_{vec_name_wo_45M[i]}.tsv', 
                                    sep = '\t', header = None).to_numpy()
    # read the data so that rows are individuals and columns are PC
    mat_PC_test[name] = np.transpose(mat_temp)

# 2. PCA

## 2.1. PCA by group (figure 2)

In [ ]:
def fn_plot_PCA(index_PC1, index_PC2,
                lower_x = None, upper_x = None, lower_y = None, upper_y = None,
                bool_save = False):
    plt.figure(figsize=(size_twocolumn, 5))

    # Plot each label with its assigned color
    for country in vec_pop_ordered:
        idx = np.array(label) == country
        plt.scatter(mat_PC['full'][idx, index_PC1 - 1],
                    mat_PC['full'][idx, index_PC2 - 1],
                    color = dict_color_subpop[country],
                    marker = dict_color_marker[country],
                    alpha = 0.7,
                    label = country,
                    s = 10)

    # Add legend and labels
    if (lower_x is not None):
        plt.xlim(lower_x, upper_x)
        plt.ylim(lower_y, upper_y)
    
    plt.xlabel(f'PC {index_PC1}', fontsize = size_font_medium)
    plt.ylabel(f'PC {index_PC2}', fontsize = size_font_medium)
    plt.legend(loc = 'center left', bbox_to_anchor = (1, 0.5), fontsize = size_font_medium)
    

    if bool_save:
        print(f'saving plot to {path_plot}')  
        plt.savefig(path_plot + 'figure2/figure2a_PCA_by_group.pdf', format = 'pdf', dpi = 1200)    
        
    plt.show()

fn_plot_PCA(1, 2, bool_save = True)

## 2.2. PCA by missing data

In [ ]:
n_col = 4
n_row = 2
name = 'train_test'
random.seed(609)
plt.rcParams['axes.facecolor'] = 'white'
list_index_PC = list(range(4)) + sorted(random.sample(range(100, 2560), 4))
list_index_PC = list(range(4)) + sorted(random.sample(range(100, 2560), 4))

fig, ax = plt.subplots(n_row, n_col, figsize = (size_twocolumn, size_twocolumn * 2/3))

gs = gridspec.GridSpec(n_row, n_col)

column_labels = ["No Missing", "90% Missing", 
                 "99% Missing", "99.9% Missing"]  # Define column labels
row_labels = ["PC1 vs PC2", "PC3 vs PC4", "PC272 vs PC816", "PC1368 vs PC2410"]  # Define row labels

vec_sign_PC_test = [1, 1, 1, 1]

for i_row in range(n_row):
    i_x = list_index_PC[i_row * 2]
    i_y = list_index_PC[i_row * 2 + 1]
    for i_col in range(n_col):
        ax = plt.subplot(gs[i_row, i_col])
        scatter_train = ax.scatter(mat_PC[name][:, i_x], 
                                   mat_PC[name][:, i_y], **para_scatter_train,
                                   s = 5)
        scatter_test = ax.scatter(mat_PC_test[vec_name_45M[i_col]][:, i_x] * vec_sign_PC_test[i_x], 
                                  mat_PC_test[vec_name_45M[i_col]][:, i_y] * vec_sign_PC_test[i_y],
                                  **para_scatter_test,
                                  s = 5)
        
        ax.tick_params(axis='both', which='major', labelsize=5)
        
        # Setting row and column labels
        if i_row == 0:
            ax.set_title(column_labels[i_col], fontsize = size_font_medium)
        if i_col == 0:
            ax.set_ylabel(row_labels[i_row], rotation=90, fontsize = size_font_medium)
            
plt.tight_layout()
plt.subplots_adjust(top = 0.85)

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right', 
           title_fontsize = size_font_medium,
           fontsize = size_font_medium)

print(f'saving plot to {path_plot}')
fig.savefig(path_plot + 'sfigure2/sfigure2_PCA_missing_data.pdf')
plt.show()